<a href="https://colab.research.google.com/github/RogerHeederer/NLP_entry/blob/master/S2S_re.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import urllib3
import zipfile
import shutil
import os
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
http = urllib3.PoolManager()
url = 'http://www.manythings.org/anki/fra-eng.zip'
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)
with http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:
  shutil.copyfileobj(r, out_file)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
  zip_ref.extractall(path)

In [3]:
lines = pd.read_csv('fra.txt', names=['src', 'tar', 'lic'], sep='\t')
del lines['lic']
len(lines)

178009

In [4]:
lines = lines.loc[:, 'src':'tar']
lines = lines[0:60000] # 6만개만 저장
lines.sample(10)

,src,tar
9782,You are morons.,Vous êtes des idiots.
59731,You need to hit the gym.,Il vous faut aller à la gym.
18685,He is watching TV.,Il regarde la TV.
42059,I have my own bedroom.,J'ai ma propre chambre.
37670,She used to hate him.,Elle le détestait.
15284,I was overweight.,J'étais en surpoids.
29502,I could use a drink.,J'aurais bien besoin d'un verre.
28126,You're very clever.,Vous êtes fort habile.
34820,He missed his flight.,Il a raté son vol.
54445,He was knee deep in mud.,Il était dans la boue jusqu'aux genoux.


In [5]:
lines.tar = lines.tar.apply(lambda x : '\t ' + x + ' \n')
lines.sample(10)

,src,tar
56948,My father was a teacher.,\t Mon père était enseignant. \n
10449,Everyone danced.,\t Tout le monde dansait. \n
47177,Do you write in French?,\t Écris-tu en français ? \n
50116,Let's begin on page 30.,\t Commençons à la page trente. \n
55300,I hope you're not alone.,\t J'espère que vous n'êtes pas seules. \n
53033,You need to stay awake.,\t Tu dois rester éveillée. \n
15982,Lower your voice.,\t Baissez la voix. \n
1504,Is he tall?,\t Est-il grand ? \n
17750,You're a man now.,\t Tu es désormais un homme. \n
46164,You don't have to lie.,\t Tu n'es pas forcé de mentir. \n


In [6]:
#글자 집합 구축
src_vocab=set()
for line in lines.src: # 1라인씩 읽어오고
  for char in line: #1라인에서 1글자씩 읽어옴
    src_vocab.add(char)

tar_vocab=set()
for line in lines.tar:
  for char in line:
    tar_vocab.add(char)

In [7]:
print(src_vocab)
print(tar_vocab)

{'J', 'n', 'u', 's', '"', 'x', 'I', '/', 'K', '9', 'z', 'X', '5', 'e', 'P', 'f', '7', 'k', 'y', 'h', 'm', '.', '8', 'q', 'A', "'", 'L', '’', 'O', 'H', 'U', 'a', 'v', 'B', 'p', 'M', 'l', '2', '$', '€', '-', 'R', 'W', 'é', 'F', '1', 'Q', 'g', 'V', '3', 'D', '?', 'i', '0', 'Z', 't', 'E', '!', '4', 'b', 'G', ',', ' ', 'C', 'r', '6', 'c', 'w', 'd', 'T', 'j', ':', 'N', '&', 'Y', '%', 'S', 'o'}
{'n', 'à', 'x', 'K', 'ù', ')', 'f', 'm', '’', 'L', 'Ô', 'p', 'é', 'W', 'F', 'Q', 'D', '?', 'E', '\u202f', 'd', 'T', ':', 'С', 'B', 's', 'X', 'P', 'k', 'h', '.', 'O', '\u2009', 'a', 'v', '‘', 'R', 'i', 'â', 'œ', '4', 'À', 'Ê', ' ', 'r', 'c', 'N', '»', 'j', '"', '9', 'e', '8', 'q', 'A', "'", 'ê', 'û', 'H', 'U', 'M', '$', '-', 'g', 'î', '\xa0', '3', '0', 'Z', 'b', 'ë', 'G', 'è', 'C', '6', '&', 'Y', '%', 'J', 'u', 'I', 'z', '5', '7', 'É', '(', 'y', '\u200b', 'ô', '«', 'Ç', 'l', '2', '\n', 'ç', '1', 'V', '\t', 't', '!', ',', 'w', 'ï', 'S', 'o'}


In [8]:
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
print(src_vocab_size)
print(tar_vocab_size)

79
106


In [9]:
#각 글자에 인덱스 부여
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])

print(src_to_index)
print(tar_to_index)

{'J': 1, 'n': 2, 'u': 3, 's': 4, '"': 5, 'x': 6, 'I': 7, '/': 8, 'K': 9, '9': 10, 'z': 11, 'X': 12, '5': 13, 'e': 14, 'P': 15, 'f': 16, '7': 17, 'k': 18, 'y': 19, 'h': 20, 'm': 21, '.': 22, '8': 23, 'q': 24, 'A': 25, "'": 26, 'L': 27, '’': 28, 'O': 29, 'H': 30, 'U': 31, 'a': 32, 'v': 33, 'B': 34, 'p': 35, 'M': 36, 'l': 37, '2': 38, '$': 39, '€': 40, '-': 41, 'R': 42, 'W': 43, 'é': 44, 'F': 45, '1': 46, 'Q': 47, 'g': 48, 'V': 49, '3': 50, 'D': 51, '?': 52, 'i': 53, '0': 54, 'Z': 55, 't': 56, 'E': 57, '!': 58, '4': 59, 'b': 60, 'G': 61, ',': 62, ' ': 63, 'C': 64, 'r': 65, '6': 66, 'c': 67, 'w': 68, 'd': 69, 'T': 70, 'j': 71, ':': 72, 'N': 73, '&': 74, 'Y': 75, '%': 76, 'S': 77, 'o': 78}
{'n': 1, 'à': 2, 'x': 3, 'K': 4, 'ù': 5, ')': 6, 'f': 7, 'm': 8, '’': 9, 'L': 10, 'Ô': 11, 'p': 12, 'é': 13, 'W': 14, 'F': 15, 'Q': 16, 'D': 17, '?': 18, 'E': 19, '\u202f': 20, 'd': 21, 'T': 22, ':': 23, 'С': 24, 'B': 25, 's': 26, 'X': 27, 'P': 28, 'k': 29, 'h': 30, '.': 31, 'O': 32, '\u2009': 33, 'a': 34

In [10]:
#인덱스가 부여된 글자집합 기준으로 훈련데이터 정수 인코딩 수행 - 영어문장
encoder_input = []
for line in lines.src:
  temp_X = []
  for w in line:
    temp_X.append(src_to_index[w])
  encoder_input.append(temp_X)
print(encoder_input[:5])

[[61, 78, 22], [30, 53, 22], [30, 53, 22], [42, 3, 2, 58], [42, 3, 2, 58]]


In [11]:
#인덱스가 부여된 글자집합 기준으로 훈련데이터 정수 인코딩 수행 - 프랑스어
decoder_input = []
for line in lines.tar:
  temp_X = []
  for w in line:
    temp_X.append(tar_to_index[w])
  decoder_input.append(temp_X)
print(decoder_input[:5])

[[98, 44, 97, 34, 44, 100, 44, 94], [98, 44, 104, 34, 92, 80, 99, 44, 100, 44, 94], [98, 44, 104, 34, 92, 80, 99, 31, 44, 94], [98, 44, 74, 105, 80, 45, 26, 20, 100, 44, 94], [98, 44, 74, 105, 80, 45, 52, 82, 20, 100, 44, 94]]


In [12]:
#디코더의 출력값이랑 비교될 실제값 정수 인코딩
decoder_target = []
for line in lines.tar:
  t=0
  temp_X = []
  for w in line:
    if t>0: # 실제값은 앞에 붙은 \t를 빼줘야함
      temp_X.append(tar_to_index[w])
    t=t+1
  decoder_target.append(temp_X)
print(decoder_target[:5])

[[44, 97, 34, 44, 100, 44, 94], [44, 104, 34, 92, 80, 99, 44, 100, 44, 94], [44, 104, 34, 92, 80, 99, 31, 44, 94], [44, 74, 105, 80, 45, 26, 20, 100, 44, 94], [44, 74, 105, 80, 45, 52, 82, 20, 100, 44, 94]]


In [13]:
#앞서 부여된 디코더값의 /t index = 98 값은 타겟(정답) 데이터 셋에서는 제거된 것을 확인
#이제 모든 데이터에 대해 정수 인코딩은 끝났으니, 패딩 작업 수행

max_src_len = max([ len(line) for line in lines.src ])
max_tar_len = max([ len(line) for line in lines.tar ])
print(max_src_len)
print(max_tar_len)

24
76


In [14]:
#영어 길이는 전부 24로 패딩, 프랑스어 길이는 전부 76으로 패딩

encoder_input = pad_sequences(encoder_input, maxlen = max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [15]:
print(encoder_input[1])

[30 53 22  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


In [16]:
#원핫벡터 적용
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

In [17]:
print(encoder_input[1][0]) # 23

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]


In [18]:
print(encoder_input[1][1]) # 31

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]


In [19]:
print(encoder_input[1][2]) # 10

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]


## S2S 훈련

In [20]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [21]:
encoder_inputs = Input(shape=(None, src_vocab_size)) #None, 79
encoder_lstm = LSTM(units=256, return_state = True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c] #은닉 값과, 셀 상태 받음

In [22]:
decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
#디코더의 첫 상태를 인코더의 은닉상태로 세팅
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states) 

decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=50, validation_split=0.2)

Epoch 1/50
750/750 [==============================] - 9s 13ms/step - loss: 0.7720 - val_loss: 0.6832
Epoch 2/50
750/750 [==============================] - 9s 12ms/step - loss: 0.4723 - val_loss: 0.5428
Epoch 3/50
750/750 [==============================] - 9s 12ms/step - loss: 0.3924 - val_loss: 0.4761
Epoch 4/50
750/750 [==============================] - 9s 12ms/step - loss: 0.3488 - val_loss: 0.4434
Epoch 5/50
750/750 [==============================] - 9s 12ms/step - loss: 0.3200 - val_loss: 0.4177
Epoch 6/50
750/750 [==============================] - 9s 12ms/step - loss: 0.2992 - val_loss: 0.4014
Epoch 7/50
750/750 [==============================] - 9s 12ms/step - loss: 0.2829 - val_loss: 0.3877
Epoch 8/50
750/750 [==============================] - 9s 12ms/step - loss: 0.2697 - val_loss: 0.3769
Epoch 9/50
750/750 [==============================] - 9s 12ms/step - loss: 0.2586 - val_loss: 0.3697
Epoch 10/50
750/750 [==============================] - 9s 12ms/step - loss: 0.2491 - val_lo

In [27]:
encoder_inputs, encoder_states

(<tf.Tensor 'input_1:0' shape=(None, None, 79) dtype=float32>,
 [<tf.Tensor 'lstm/PartitionedCall:2' shape=(None, 256) dtype=float32>,
  <tf.Tensor 'lstm/PartitionedCall:3' shape=(None, 256) dtype=float32>])

##S2S 동작시키기##

1. 번역하고자 하는 입력 문장이 인코더에 먼저 들어가서 은닉 상태와 셀 상태를 얻어온다
2. 상태값과 <sos>를 디코더로 보낸다
3. 디코더는 <eos> 가 나올 때 까지 다음 문자를 예측하는 행동을 반복한다

In [28]:
#훈련 과정에서 사용한 인코더 인풋값들과 상태값을 재사용
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [29]:
encoder_model

In [31]:
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state = decoder_states_inputs)

decoder_states = [state_h, state_c]

decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [32]:
#인덱스로부터 단어를 얻는 것 구현
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

Decode_sequence 함수가 이해가 잘 안되네

In [33]:

def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, tar_to_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_tar_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, tar_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [35]:
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.src[seq_index])
    print('정답 문장:', lines.tar[seq_index][1:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력

-----------------------------------
입력 문장: hi, where is the tennis court
정답 문장:  Cours ! 
번역기가 번역한 문장:  Cours ! 
-----------------------------------
입력 문장: hi, where is the tennis court
정답 문장:  Je suis partie. 
번역기가 번역한 문장:  Je suis parti. 
-----------------------------------
입력 문장: hi, where is the tennis court
정답 문장:  Appelez-nous ! 
번역기가 번역한 문장:  Appelle-nous ! 
-----------------------------------
입력 문장: hi, where is the tennis court
정답 문장:  Comme c'est gentil ! 
번역기가 번역한 문장:  Comme c'est chouette ! 
-----------------------------------
입력 문장: hi, where is the tennis court
정답 문장:  Tourne à gauche. 
번역기가 번역한 문장:  Éteins-le ! 
